# Baseline Kernel Methods & Random Fourier Features

In this notebook we compare the performance of an exact kernel classifier with a kernel approximation using Random Fourier Features (RFF) on the Sign Language MNIST dataset. We will assess the methods in terms of accuracy, training time, prediction time, and memory usage. Later sections explore scaling behavior and extrapolate the computational requirements for applying these methods to the full dataset.

In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)
sns.set_style('whitegrid')

# Ignore warnings for clarity
warnings.filterwarnings('ignore')

# Add project root directory to the path
sys.path.append('..')

# Import our modules
from kernel_methods.utils.data_loader import get_sample_dataset, load_processed_data
from kernel_methods.models.kernel_base import KernelClassifier
from kernel_methods.models.kernel_approximation_classifier import KernelApproximationClassifier


# For reproducibility
np.random.seed(42)

print('Imports successful')

## 1. Load the Data

We begin by loading a small sample of the Sign Language MNIST dataset for quick experimentation. We will print the shapes of the training and validation sets along with the class distribution.

In [ ]:
# Load a sample of the data
X_train_sample, X_val_sample, y_train_sample, y_val_sample = get_sample_dataset(n_samples=500, random_state=42)

print(f"Training sample shape: {X_train_sample.shape}")
print(f"Validation sample shape: {X_val_sample.shape}")

print("\nClass distribution in training sample:")
unique, counts = np.unique(y_train_sample, return_counts=True)
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples ({count/len(y_train_sample)*100:.1f}%)")

## 2. Baseline: Exact Kernel Method

We now train an exact kernel classifier using an RBF kernel. This classifier computes the full kernel matrix which becomes expensive for large datasets. The performance of this exact method will be our baseline.

In [ ]:
# Initialize the exact kernel classifier
kernel_clf = KernelClassifier(kernel='rbf', gamma=0.01, C=1.0)

# Train the classifier
start_time = time.time()
kernel_clf.fit(X_train_sample, y_train_sample)
train_time = time.time() - start_time

# Predict on the validation set
start_time = time.time()
y_val_pred = kernel_clf.predict(X_val_sample)
predict_time = time.time() - start_time

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val_sample, y_val_pred)

# Log results
exact_metrics = kernel_clf.get_performance_metrics()
print(f"Exact Kernel Method Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Training time: {train_time:.4f} seconds")
print(f"Prediction time: {predict_time:.4f} seconds")
print(f"Estimated memory usage: {exact_metrics['memory_usage_mb']:.2f} MB")

### Visualize the Confusion Matrix

A confusion matrix helps us better understand the classification performance. We use a heatmap to visualize the normalized confusion matrix.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def plot_confusion_matrix(y_true, y_pred, class_names=None, figsize=(14, 12), title='Confusion Matrix'):
    """
    Plot a confusion matrix with clear boundary lines between cells.
    
    Parameters:
    -----------
    y_true : array-like
        Ground truth (correct) target values.
    y_pred : array-like
        Estimated targets as returned by a classifier.
    class_names : list, optional
        List of class names for axis labels.
    figsize : tuple, optional
        Figure size (width, height) in inches.
    title : str, optional
        Title of the confusion matrix plot.
    """
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Normalize the confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Create figure and axes
    plt.figure(figsize=figsize)
    
    # Generate class names if not provided
    if class_names is None:
        class_names = [str(i) for i in range(len(np.unique(np.concatenate([y_true, y_pred]))))]
    
    # Create prettier plot with seaborn - important changes for boundary lines:
    # 1. Increased linewidths parameter (from 0.5 to 1.0)
    # 2. Added linecolor parameter to make boundaries more visible
    ax = sns.heatmap(
        cm_normalized, 
        annot=True,
        fmt='.2f',
        cmap='Blues',
        square=True,
        xticklabels=class_names,
        yticklabels=class_names,
        vmin=0,
        vmax=1,
        linewidths=1.0,         # Thicker lines between cells
        linecolor='black',      # Black boundary lines for contrast
        cbar_kws={"shrink": .8}
    )
    
    # Add a strong outer border to the entire matrix
    for _, spine in ax.spines.items():
        spine.set_visible(True)
        spine.set_linewidth(2)
        spine.set_color('black')
    
    # Improve aesthetics
    plt.title(title, fontsize=16, pad=20)
    plt.ylabel('True Label', fontsize=14, labelpad=10)
    plt.xlabel('Predicted Label', fontsize=14, labelpad=10)
    
    # Rotate the tick labels and set alignment
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontweight='bold')
    plt.setp(ax.get_yticklabels(), fontweight='bold')
    
    # Tight layout to ensure everything fits
    plt.tight_layout()
    
    # Show plot
    plt.show()
    
    # Also return the raw confusion matrix for further analysis if needed
    return cm

alphabet = [chr(i+65) for i in range(26) if chr(i+65) not in ['J', 'Z']]

# Plot the improved confusion matrix
plot_confusion_matrix(
    y_val_sample, 
    y_val_pred,
    class_names=alphabet,
    title='Sign Language Recognition - Exact Kernel Method'
)

#### Strengths of the Model
- Perfect Classification for several letters:
    - B, N, P, and V (1.00 accuracy)
    - Letters with distinctive hand shapes show the strongest performance
- Strong Performance on:
    - Letter T (0.88)
    - Letter Q (0.83)
    - Letters C, H, R, Y (0.67)
    - Letter L (0.75)

#### Key Challenges
- Complete Misclassification of letter A (0.0 on diagonal)
- Poor Recognition of:
    - Letter E (scattered across multiple classes)
    - Letter U (distributed across several predictions)
    - Letter W (fragmented across many classes)
    - Letter X (appears missing from diagonal)

The confusion patterns strongly correlate with visual similarities in ASL hand shapes:
- Letters with similar finger positions or orientations are frequently confused
- Letters with unique, distinctive shapes (B, N, P, V) achieve perfect recognition
- Letters requiring subtle distinctions show higher confusion rates

## 3. Random Fourier Features Approximation

Next, we use Random Fourier Features (RFF) to approximate the RBF kernel. This transformation maps the data into a lower-dimensional space where an inner product approximates the RBF kernel. We test several component sizes to explore the accuracy–performance trade-off.

In [ ]:
# List of RFF component sizes to try
component_sizes = [50, 100, 200, 500]

# Store RFF results
rff_results = []

for n_components in component_sizes:
    print(f"\nTraining RFF with {n_components} components...")
    
    # Initialize RFF classifier with kernel approximation
    rff_clf = KernelApproximationClassifier(
        approximation='rff',
        n_components=n_components,
        gamma=0.01,
        C=1.0,
        random_state=42
    )
    
    # Train classifier
    start_time = time.time()
    rff_clf.fit(X_train_sample, y_train_sample)
    train_time_rff = time.time() - start_time
    
    # Evaluate classifier
    start_time = time.time()
    y_val_pred_rff = rff_clf.predict(X_val_sample)
    predict_time_rff = time.time() - start_time
    
    # Compute accuracy
    acc_rff = accuracy_score(y_val_sample, y_val_pred_rff)
    
    # Estimate memory usage for RFF (approximation formula)
    memory_usage_rff = (X_train_sample.shape[1] * n_components * 8) / (1024 * 1024)  # in MB
    
    print(f"Accuracy: {acc_rff:.4f}")
    print(f"Training time: {train_time_rff:.4f} seconds")
    print(f"Prediction time: {predict_time_rff:.4f} seconds")
    print(f"Estimated memory usage: {memory_usage_rff:.2f} MB")
    
    rff_results.append({
        'n_components': n_components,
        'accuracy': acc_rff,
        'train_time': train_time_rff,
        'predict_time': predict_time_rff,
        'memory_usage': memory_usage_rff,
        'predictions': y_val_pred_rff
    })

### Visualize Confusion Matrix for the Best RFF Model

We now determine which RFF configuration achieved the highest accuracy and plot its confusion matrix.

In [ ]:
# Find the best RFF model based on validation accuracy
best_rff_idx = np.argmax([result['accuracy'] for result in rff_results])
best_rff_model = rff_results[best_rff_idx]
best_components = best_rff_model['n_components']
best_predictions = best_rff_model['predictions']

# Create the alphabet list (A-X, excluding J and Z)
alphabet = [chr(i+65) for i in range(26) if chr(i+65) not in ['J', 'Z']]

# Plot the confusion matrix for RFF method
plot_confusion_matrix(
    y_val_sample, 
    best_predictions,
    class_names=alphabet,
    title=f'Sign Language Recognition - RFF Method ({best_components} components)'
)

#### Insights on the Confusion matrix
1. Strong performers: Letters B, K, N, and V have perfect (1.00) classification accuracy, indicating the RFF method works extremely well for these signs.
2. Moderate performers: Letters like L (0.75), Y (0.67), C (0.67), and T (0.62) have good but not perfect accuracy.
3. Challenging signs: Many letters show poor classification, particularly E, I, G, H, M, and W, which are frequently confused with other letters.
4. Specific confusion patterns:
    - Letter A is frequently confused with C and P
    - Letter D is confused with K (0.50)
    - Letter W is often misclassified as K (0.36)
    - Letter G is split between being classified as G, F, and S (0.33 each)
5. Systematic issues: Some letters appear to have similar hand shapes that the RFF method struggles to differentiate, suggesting that either more components or a different kernel approach might be needed.

## 4. Performance Comparison

We now compile the results of the exact kernel method and the various RFF approximations into a comparison table and visualize the differences in accuracy, training time, prediction time, and memory usage.

In [ ]:
# Build comparison DataFrame
comparison_data = {
    'Method': ['Exact Kernel'] + [f'RFF-{res["n_components"]}' for res in rff_results],
    'Accuracy': [accuracy] + [res['accuracy'] for res in rff_results],
    'Training Time (s)': [train_time] + [res['train_time'] for res in rff_results],
    'Prediction Time (s)': [predict_time] + [res['predict_time'] for res in rff_results],
    'Memory Usage (MB)': [exact_metrics['memory_usage_mb']] + [res['memory_usage'] for res in rff_results]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df)

# Plot performance comparisons
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

# Accuracy
sns.barplot(x='Method', y='Accuracy', data=comparison_df, ax=axes[0])
axes[0].set_title('Accuracy Comparison')
axes[0].set_ylim(0, 1)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)

# Training time
sns.barplot(x='Method', y='Training Time (s)', data=comparison_df, ax=axes[1])
axes[1].set_title('Training Time Comparison')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45)

# Prediction time
sns.barplot(x='Method', y='Prediction Time (s)', data=comparison_df, ax=axes[2])
axes[2].set_title('Prediction Time Comparison')
axes[2].set_xticklabels(axes[2].get_xticklabels(), rotation=45)

# Memory usage
sns.barplot(x='Method', y='Memory Usage (MB)', data=comparison_df, ax=axes[3])
axes[3].set_title('Memory Usage Comparison')
axes[3].set_xticklabels(axes[3].get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()

#### Insights:

1. Accuracy trade-offs:
   - The exact kernel method significantly outperforms all RFF approximations (~50% vs 20-33%)
   - Among RFF variants, RFF-200 performs best accuracy-wise, with diminishing returns at RFF-500
   - There's a substantial accuracy cost when using the approximation methods

2. Computational efficiency:
   - Training time: RFF-100 is fastest, while exact kernel and RFF-500 are similarly slow
   - Prediction time: All RFF variants are faster than the exact kernel, with RFF-100 being fastest
   - Memory usage: RFF methods use significantly less memory (RFF-50 uses only ~6% of exact kernel's memory)

3. Scaling characteristics:
   - Adding more components generally increases accuracy up to a point (200 components)
   - Higher component counts directly increase memory usage and training time
   - The efficiency benefits of RFF diminish at 500 components

4. Optimal configurations:
   - RFF-100 offers the best prediction speed
   - RFF-200 provides the best balance between accuracy and resource usage
   - The exact kernel should be used when accuracy is the only priority

This demonstrates the fundamental trade-off in kernel methods: exact computation provides better accuracy but at higher computational cost, while approximation methods sacrifice some accuracy for significant gains in speed and memory efficiency.


## 5. Scaling Analysis

To investigate scalability, we run experiments on subsets of increasing size. This analysis reveals how the training time, prediction time, memory usage, and accuracy change as the number of samples increases.

In [ ]:
from tqdm import tqdm

# Define sample sizes for scaling analysis
sample_sizes = [100, 200, 500, 1000, 2000]
scaling_results = []

# Use best RFF component setting from previous experiments
best_n_components = rff_results[best_rff_idx]['n_components']

for size in tqdm(sample_sizes, desc="Testing different sample sizes"):
    print(f"\nTesting with sample size: {size}")
    
    # Get a subset of data
    X_train_subset, X_val_subset, y_train_subset, y_val_subset = get_sample_dataset(n_samples=size, random_state=42)
    result = {'sample_size': size}
    
    # Test exact kernel method
    try:
        exact_clf = KernelClassifier(kernel='rbf', gamma=0.01, C=1.0)
        start_time = time.time()
        exact_clf.fit(X_train_subset, y_train_subset)
        exact_train_time = time.time() - start_time
        
        start_time = time.time()
        y_pred_exact = exact_clf.predict(X_val_subset)
        exact_predict_time = time.time() - start_time
        
        acc_exact = accuracy_score(y_val_subset, y_pred_exact)
        exact_metrics = exact_clf.get_performance_metrics()
        
        result.update({
            'exact_train_time': exact_train_time,
            'exact_predict_time': exact_predict_time,
            'exact_accuracy': acc_exact,
            'exact_memory': exact_metrics['memory_usage_mb']
        })
        
        print(f"Exact Kernel - Accuracy: {acc_exact:.4f}, Training time: {exact_train_time:.4f}s")
    except Exception as e:
        print(f"Error with exact kernel for size {size}: {e}")
        result.update({
            'exact_train_time': float('nan'),
            'exact_predict_time': float('nan'),
            'exact_accuracy': float('nan'),
            'exact_memory': float('nan')
        })
    
    # Test RFF approximation
    rff_clf = KernelApproximationClassifier(
        approximation='rff',
        n_components=best_n_components,
        gamma=0.01,
        C=1.0,
        random_state=42
    )
    start_time = time.time()
    rff_clf.fit(X_train_subset, y_train_subset)
    rff_train_time = time.time() - start_time
    
    start_time = time.time()
    y_pred_rff = rff_clf.predict(X_val_subset)
    rff_predict_time = time.time() - start_time
    
    acc_rff = accuracy_score(y_val_subset, y_pred_rff)
    rff_memory = (X_train_subset.shape[1] * best_n_components * 8) / (1024 * 1024)
    
    result.update({
        'rff_train_time': rff_train_time,
        'rff_predict_time': rff_predict_time,
        'rff_accuracy': acc_rff,
        'rff_memory': rff_memory
    })
    
    print(f"RFF ({best_n_components} components) - Accuracy: {acc_rff:.4f}, Training time: {rff_train_time:.4f}s")
    
    scaling_results.append(result)

scaling_df = pd.DataFrame(scaling_results)
print(scaling_df)

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

# Training time
axes[0].set_title('Training Time vs Dataset Size')
axes[0].set_xlabel('Number of Training Samples')
axes[0].set_ylabel('Training Time (s)')
valid_exact = ~np.isnan(scaling_df['exact_train_time'])
axes[0].plot(scaling_df['sample_size'][valid_exact], scaling_df['exact_train_time'][valid_exact], 'o-', label='Exact Kernel')
axes[0].plot(scaling_df['sample_size'], scaling_df['rff_train_time'], 'o-', label=f'RFF ({best_n_components} components)')
axes[0].legend()
axes[0].grid(True)

# Prediction time
axes[1].set_title('Prediction Time vs Dataset Size')
axes[1].set_xlabel('Number of Training Samples')
axes[1].set_ylabel('Prediction Time (s)')
valid_exact_pred = ~np.isnan(scaling_df['exact_predict_time'])
axes[1].plot(scaling_df['sample_size'][valid_exact_pred], scaling_df['exact_predict_time'][valid_exact_pred], 'o-', label='Exact Kernel')
axes[1].plot(scaling_df['sample_size'], scaling_df['rff_predict_time'], 'o-', label=f'RFF ({best_n_components} components)')
axes[1].legend()
axes[1].grid(True)

# Memory usage
axes[2].set_title('Memory Usage vs Dataset Size')
axes[2].set_xlabel('Number of Training Samples')
axes[2].set_ylabel('Memory Usage (MB)')
valid_exact_mem = ~np.isnan(scaling_df['exact_memory'])
axes[2].plot(scaling_df['sample_size'][valid_exact_mem], scaling_df['exact_memory'][valid_exact_mem], 'o-', label='Exact Kernel')
axes[2].plot(scaling_df['sample_size'], scaling_df['rff_memory'], 'o-', label=f'RFF ({best_n_components} components)')
axes[2].legend()
axes[2].grid(True)

# Accuracy
axes[3].set_title('Accuracy vs Dataset Size')
axes[3].set_xlabel('Number of Training Samples')
axes[3].set_ylabel('Accuracy')
valid_exact_acc = ~np.isnan(scaling_df['exact_accuracy'])
axes[3].plot(scaling_df['sample_size'][valid_exact_acc], scaling_df['exact_accuracy'][valid_exact_acc], 'o-', label='Exact Kernel')
axes[3].plot(scaling_df['sample_size'], scaling_df['rff_accuracy'], 'o-', label=f'RFF ({best_n_components} components)')
axes[3].legend()
axes[3].grid(True)

plt.tight_layout()
plt.show()

#### Insights:
1. Computational scaling:
   - Exact kernel exhibits quadratic scaling (O(n²)) for training time, increasing dramatically from 0.5s to 3.1s as samples grow
   - RFF training time remains nearly constant (around 0.1s) regardless of dataset size
   - At 2000 samples, exact kernel is ~30× slower than RFF for training

2. Memory efficiency:
   - Exact kernel memory usage grows quadratically, reaching ~43MB at 2000 samples
   - RFF maintains effectively constant memory usage (~1MB) across all dataset sizes
   - The memory gap becomes extreme at scale (>40× difference at 2000 samples)

3. Prediction performance:
   - Exact kernel prediction time scales linearly with dataset size
   - RFF prediction time increases very slowly, maintaining a ~5× speed advantage at 2000 samples
   - This prediction efficiency becomes critical for real-time applications

4. Accuracy-efficiency tradeoff:
   - Both methods show improved accuracy with more training data
   - The exact kernel maintains a consistent accuracy advantage (~20%)
   - The accuracy gap doesn't widen with scale, suggesting RFF's performance disadvantage is stable

This analysis demonstrates why RFF is valuable for large-scale applications: while sacrificing some accuracy, it provides massive scalability benefits that make kernel methods practical for large datasets where exact kernel calculations would be prohibitively expensive.


## 6. Extrapolation to Full Dataset

Finally, we estimate the computational requirements for training on the full Sign Language MNIST dataset. We use a quadratic fit for the exact kernel method and a linear fit for the RFF approximation to predict training time and memory usage.

In [ ]:
# Load full dataset to get the total number of samples (without loading entire data into memory for this demo)
X_train_full, _, _, _, _, _ = load_processed_data()
full_dataset_size = len(X_train_full)
print(f"Full training dataset size: {full_dataset_size} samples")

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

def estimate_for_full_dataset(df):
    # Estimate for exact kernel method using a quadratic fit
    valid_exact = ~np.isnan(df['exact_train_time'])
    if sum(valid_exact) >= 3:
        X_poly = PolynomialFeatures(degree=2).fit_transform(df['sample_size'][valid_exact].values.reshape(-1, 1))
        model_quad = LinearRegression().fit(X_poly, df['exact_train_time'][valid_exact])
        X_full_poly = PolynomialFeatures(degree=2).fit_transform(np.array([[full_dataset_size]]))
        exact_train_time_est = model_quad.predict(X_full_poly)[0]
        
        model_mem = LinearRegression().fit(X_poly, df['exact_memory'][valid_exact])
        exact_memory_est = model_mem.predict(X_full_poly)[0]
        print(f"Estimated resources for exact kernel method on full dataset ({full_dataset_size} samples):")
        print(f"  Training time: {exact_train_time_est:.2f} seconds ({exact_train_time_est/60:.2f} minutes)")
        print(f"  Memory usage: {exact_memory_est:.2f} MB ({exact_memory_est/1024:.2f} GB)")
    else:
        exact_train_time_est = float('nan')
        exact_memory_est = float('nan')
        print("Not enough data points to estimate exact kernel method resources.")
        
    # Estimate for RFF (linear scaling assumed)
    model_rff = LinearRegression().fit(df['sample_size'].values.reshape(-1, 1), df['rff_train_time'])
    rff_train_time_est = model_rff.predict([[full_dataset_size]])[0]
    
    model_rff_mem = LinearRegression().fit(df['sample_size'].values.reshape(-1, 1), df['rff_memory'])
    rff_memory_est = model_rff_mem.predict([[full_dataset_size]])[0]
    
    print(f"\nEstimated resources for RFF ({best_n_components} components) on full dataset ({full_dataset_size} samples):")
    print(f"  Training time: {rff_train_time_est:.2f} seconds ({rff_train_time_est/60:.2f} minutes)")
    print(f"  Memory usage: {rff_memory_est:.2f} MB ({rff_memory_est/1024:.2f} GB)")
    
    return {
        'exact_train_time_est': exact_train_time_est,
        'exact_memory_est': exact_memory_est,
        'rff_train_time_est': rff_train_time_est,
        'rff_memory_est': rff_memory_est
    }

estimates = estimate_for_full_dataset(scaling_df)


This extrapolation analysis demonstrates the dramatic real-world impact of algorithmic scaling properties:

1. Training efficiency contrast:
   - Exact kernel: 8.86 minutes (531.66 seconds)
   - RFF (200): 0.02 minutes (1.04 seconds)
   - **512× speed improvement** with approximation

2. Memory requirements:
   - Exact kernel: 3.73 GB (3816.12 MB)
   - RFF (200): 0.001 GB (1.20 MB)
   - **3,180× memory reduction** with approximation

3. Practical implications:
   - The exact kernel method is on the edge of feasibility for standard hardware
   - RFF transforms a substantial computational problem into a trivial one
   - The exact method would struggle with even slightly larger datasets

4. Scaling relationships:
   - The 11× increase in data size (2,000 → 21,964 samples) caused:
     - ~171× increase in exact kernel training time 
     - ~97× increase in exact kernel memory usage
   - Confirming worse-than-linear (quadratic) scaling behavior

This analysis makes clear why kernel approximation methods are essential for practical machine learning applications, enabling the use of kernel-based approaches on datasets that would otherwise be completely impractical to process using exact methods.
